# imports

In [1]:
import numpy as np
import pandas as pd

# Generate new fitting script from template

In [2]:
#There are three burst where not all detectors are on
#modify the fitting script by hand, since the data grouping will be different

def fit_script(HEDfname=None,KW1fname=None,KW2fname=None,GBMn1fname=None,GBMn2fname=None,GBMb1fname=None,outfname=None):
    
    fnames = [HEDfname,KW1fname,KW2fname,GBMn1fname,GBMn2fname,GBMb1fname,outfname]
    if all(isinstance(fname, str) for fname in fnames) == False:
        return(print("input must be string"))
    
    with open('fitting.xcm',"r") as f:
        lines = f.readlines() #198 lines of code
        for i in np.arange(0,len(lines)):
            lines[i] = lines[i].replace("HED_PHA_1.fits",HEDfname)
            lines[i] = lines[i].replace("KW201227_1_gr20.pha",KW1fname)
            lines[i] = lines[i].replace("KW201227_2_gr20.pha",KW2fname)
            lines[i] = lines[i].replace("GRB201227A_n4_gr20.pha",GBMn1fname)
            lines[i] = lines[i].replace("GRB201227A_n8_gr20.pha",GBMn2fname)
            lines[i] = lines[i].replace("GRB201227A_b0_gr20.pha",GBMb1fname)
            
    with open(outfname+".xcm","w") as out:
        for line in lines:
            out.write(line)

In [3]:
fit_script(HEDfname="bla",KW1fname="blaa",KW2fname="blaaa",GBMn1fname="la",GBMn2fname="laa",GBMb1fname="laaa",outfname="fooo")

In [4]:
#GRB 190206A don't have GBM data, so i can't use the fit_script above
#I can go in an replace HED and KW files, but then i need to remove the GBM grouping and modeling that includes GBM

# Get spectral parameters from log file
    -'Current model list' starts from index 144
    -'\n' contained in every line

In [5]:
with open('template.log','r') as f:
    logLines = f.readlines()
    #looking for this line: #Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On
    for l in logLines:
        if l == '#Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On':
            print(l)

In [6]:
for i,l in enumerate(logLines):
    if l == '#Current model list:\n':
        print('model list start at index: ', i)
        modList = logLines[i:]
        break

model list start at index:  144


In [7]:
#TODO: load the spec parameters for all the datagroups
#remove '\n' from line

In [8]:
clean_modList = []
for l in modList:
    l = l.replace('\n','')
    l = l.replace('#','')
    clean_modList.append(l)

In [9]:
keys = {}
for i,l in enumerate(clean_modList):
    if 'Data group:' in l:
        key = l.strip()
        keys[key] = i
startingIndecies = np.fromiter(keys.values(), dtype=int)
n_dataGroups = len(startingIndecies)

if np.all(np.diff(startingIndecies)) == True:
    size_dataGroup = np.diff(np.fromiter(keys.values(), dtype=int))[0]
else:
    print('varying length of datagroups')

In [10]:
startingIndecies,n_dataGroups

(array([ 6, 16, 26, 36]), 4)

In [42]:
spec_data = {}
for key,value in keys.items():
    newTargetModel = []
    targetModel = clean_modList[value+1:value+size_dataGroup]
    for par in targetModel:
        newpar = par.strip()
        newpar = newpar.split()
        del newpar[1]
        newTargetModel.append(newpar)
    spec_data[key] = newTargetModel

In [18]:
for l in spec_data.values():
    del l[0][0]
    print(l[0][0])

constant
constant
constant
constant


In [43]:
list(spec_data.values())

[[['1', 'constant', 'factor', '1.27737', '+/-', '0.127300'],
  ['2', 'cflux', 'Emin', 'keV', '10.0000', 'frozen'],
  ['3', 'cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
  ['4', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
  ['5', 'ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
  ['6', 'ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
  ['7', 'ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
  ['8', 'ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
  ['9', 'ngrbep', 'norm', '1.00000', 'frozen']],
 [['10', 'constant', 'factor', '1.00000', 'frozen'],
  ['11', 'cflux', 'Emin', 'keV', '10.0000', '=', 'p2'],
  ['12', 'cflux', 'Emax', 'keV', '1.00000E+04', '=', 'p3'],
  ['13', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '=', 'p4'],
  ['14', 'ngrbep', 'alpha', '-0.185890', '=', 'p5'],
  ['15', 'ngrbep', 'beta', '-3.44617', '=', 'p6'],
  ['16', 'ngrbep', 'epeak', 'keV', '922.399', '=', 'p7'],
  ['17', 'ngrbep', 'enorm', 'keV', '100.000', '=', 'p8'],
  ['18', 'ngrbep

In [44]:
v = list(spec_data.values())
for i in range(0,4):
    for j in range(0,9):
        del v[i][j][0]
v

[[['constant', 'factor', '1.27737', '+/-', '0.127300'],
  ['cflux', 'Emin', 'keV', '10.0000', 'frozen'],
  ['cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
  ['cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
  ['ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
  ['ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
  ['ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
  ['ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
  ['ngrbep', 'norm', '1.00000', 'frozen']],
 [['constant', 'factor', '1.00000', 'frozen'],
  ['cflux', 'Emin', 'keV', '10.0000', '=', 'p2'],
  ['cflux', 'Emax', 'keV', '1.00000E+04', '=', 'p3'],
  ['cflux', 'lg10Flux', 'cgs', '-4.18518', '=', 'p4'],
  ['ngrbep', 'alpha', '-0.185890', '=', 'p5'],
  ['ngrbep', 'beta', '-3.44617', '=', 'p6'],
  ['ngrbep', 'epeak', 'keV', '922.399', '=', 'p7'],
  ['ngrbep', 'enorm', 'keV', '100.000', '=', 'p8'],
  ['ngrbep', 'norm', '1.00000', '=', 'p9']],
 [['constant', 'factor', '1.31508', '+/-', '8.29398E-02'],
  ['c

In [52]:
for i in range(0,4):
    key = list(spec_data.keys())[i]
    clean_dict[key] = v[i]

In [80]:
#Need to get the parameter names and put them where the numbers are in the df
headerName_list = []
for j in range(0,9):
    par = list(clean_dict.values())[0][j][0]
    typ = list(clean_dict.values())[0][j][1]
    header_name = par + ' ' + '[' + typ + ']'
    headerName_list.append(header_name)

In [98]:
#problem that the lists are different lengths

In [133]:
if (float('1.27737') == float) == True:
    print('true')
    

In [135]:
(type(float('1.27737')) == float)

True

In [149]:
cell_input = [['']*9]*4
cell_input

[['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '']]

In [146]:
list(clean_dict.values())[0]

[['constant', 'factor', '1.27737', '+/-', '0.127300'],
 ['cflux', 'Emin', 'keV', '10.0000', 'frozen'],
 ['cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
 ['cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
 ['ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
 ['ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
 ['ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
 ['ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
 ['ngrbep', 'norm', '1.00000', 'frozen']]

In [198]:
float('3.00161E-02')

0.0300161

In [255]:
def get_cellInput(l):
    cell_input = ''
    for x in l:
        if x == '+/-':
            cell_input += ' ' + x + ' '
        else:
            try:
                float(x)
                cell_input += str(np.round(float(x),3))
            except ValueError:
                continue
    return(cell_input)

In [273]:
#cell_input = [['']*9]*4

for i in range(0,4):
    for j in range(0,9):
        #print(list(clean_dict.values())[i][j])
        l = list(clean_dict.values())[i][j]
        cell_input[i][j] = get_cellInput(l)

In [ ]:
list(clean_dict.values())[i][j]

In [274]:
for i in range(0,4):
    print(cell_input[i][0])

1.395 +/- 0.093
1.395 +/- 0.093
1.395 +/- 0.093
1.395 +/- 0.093


In [257]:
df = pd.DataFrame(data = clean_dict).T
df

,0,1,2,3,4,5,6,7,8
Data group: 1,"[constant, factor, 1.27737, +/-, 0.127300]","[cflux, Emin, keV, 10.0000, frozen]","[cflux, Emax, keV, 1.00000E+04, frozen]","[cflux, lg10Flux, cgs, -4.18518, +/-, 3.00161E...","[ngrbep, alpha, -0.185890, +/-, 7.28931E-02]","[ngrbep, beta, -3.44617, +/-, 0.512888]","[ngrbep, epeak, keV, 922.399, +/-, 58.2440]","[ngrbep, enorm, keV, 100.000, frozen]","[ngrbep, norm, 1.00000, frozen]"
Data group: 2,"[constant, factor, 1.00000, frozen]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"
Data group: 3,"[constant, factor, 1.31508, +/-, 8.29398E-02]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"
Data group: 4,"[constant, factor, 1.39527, +/-, 9.27995E-02]","[cflux, Emin, keV, 10.0000, =, p2]","[cflux, Emax, keV, 1.00000E+04, =, p3]","[cflux, lg10Flux, cgs, -4.18518, =, p4]","[ngrbep, alpha, -0.185890, =, p5]","[ngrbep, beta, -3.44617, =, p6]","[ngrbep, epeak, keV, 922.399, =, p7]","[ngrbep, enorm, keV, 100.000, =, p8]","[ngrbep, norm, 1.00000, =, p9]"


In [258]:
for i,key in enumerate(list(df.columns)):
    df.rename({key: headerName_list[i]}, axis=1,inplace=True)

In [260]:
list(keys.keys())

['Data group: 1', 'Data group: 2', 'Data group: 3', 'Data group: 4']

In [261]:
final_dict={}
for i in range(0,4):
    final_dict[list(keys.keys())[i]] = cell_input[i]

In [262]:
DF = pd.DataFrame(final_dict).T
for i,key in enumerate(list(DF.columns)):
    DF.rename({key: headerName_list[i]}, axis=1,inplace=True)

In [267]:
cell_input[2]

['1.395 +/- 0.093',
 '10.0',
 '10000.0',
 '-4.185',
 '-0.186',
 '-3.446',
 '922.399',
 '100.0',
 '1.0']

In [264]:
DF

,constant [factor],cflux [Emin],cflux [Emax],cflux [lg10Flux],ngrbep [alpha],ngrbep [beta],ngrbep [epeak],ngrbep [enorm],ngrbep [norm]
Data group: 1,1.395 +/- 0.093,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0
Data group: 2,1.395 +/- 0.093,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0
Data group: 3,1.395 +/- 0.093,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0
Data group: 4,1.395 +/- 0.093,10.0,10000.0,-4.185,-0.186,-3.446,922.399,100.0,1.0


In [263]:
print(DF.T.style.to_latex(position_float='centering',position='h!',hrules=True,multirow_align='c'))

\begin{table}[h!]
\centering
\begin{tabular}{lllll}
\toprule
 & Data group: 1 & Data group: 2 & Data group: 3 & Data group: 4 \\
\midrule
constant [factor] & 1.395 +/- 0.093 & 1.395 +/- 0.093 & 1.395 +/- 0.093 & 1.395 +/- 0.093 \\
cflux [Emin] & 10.0 & 10.0 & 10.0 & 10.0 \\
cflux [Emax] & 10000.0 & 10000.0 & 10000.0 & 10000.0 \\
cflux [lg10Flux] & -4.185 & -4.185 & -4.185 & -4.185 \\
ngrbep [alpha] & -0.186 & -0.186 & -0.186 & -0.186 \\
ngrbep [beta] & -3.446 & -3.446 & -3.446 & -3.446 \\
ngrbep [epeak] & 922.399 & 922.399 & 922.399 & 922.399 \\
ngrbep [enorm] & 100.0 & 100.0 & 100.0 & 100.0 \\
ngrbep [norm] & 1.0 & 1.0 & 1.0 & 1.0 \\
\bottomrule
\end{tabular}
\end{table}

